In [6]:
import requests
from bs4 import BeautifulSoup

keyword = input("검색어 입력: ")
response = requests.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query="+keyword)
html = response.text

soup = BeautifulSoup(html,'html.parser')
# html에서 뉴스 제목 부분의 클래스 명
# a 태그의 정보가 리스트 형태로 담긴다!
links = soup.select(".news_tit")    

for link in links:
    title = link.text # 테그 안에 텍스트 요소를 가져옴
    url = link.attrs['href']    # href의 속성값을 가져옴
    print(title,url)

검색어 입력: 삼성전자
삼성전자, 세계 최초 '강아지용 스마트폰' 출시? http://www.edaily.co.kr/news/newspath.asp?newsid=01302166635571856
삼성전자, 비스포크 냉장고 구매 고객에 패널 50% 할인 https://www.yna.co.kr/view/AKR20230331027400003?input=1195m
'효자는 옛말' 반도체 수출 34%↓…삼성전자·SK하이닉스 셈법 복잡 http://news.tvchosun.com/site/data/html_dir/2023/04/01/2023040190056.html
삼성전자, 7일 반도체 '어닝쇼크' 유력 [전기전자 1분기 실적 명암①] http://www.newsis.com/view/?id=NISX20230331_0002249575&cID=13001&pID=13000
삼성전자 현직 부사장, 로봇 업체 레인보우로보틱스 합류 https://biz.chosun.com/it-science/ict/2023/03/31/VBQH5TJILJCMHOWVDKM6QC6ITQ/?utm_source=naver&utm_medium=original&utm_campaign=biz
삼성전자 "비스포크 냉장고 사면 패널 절반값에 드려요" https://www.news1.kr/articles/5000091
'역시 기름집'...삼성전자보다 작년 연봉 높아 http://www.fnnews.com/news/202303311738098007
일반 직원 평균연봉, 삼성전자보다 높은 곳은 어디? https://www.nocutnews.co.kr/news/5918634
"예비신랑, 삼성전자에 물렸습니다"…결혼 앞둔 여성들 '멘붕' [박의명의 불개... https://www.hankyung.com/finance/article/202304018991i
반도체 후공정 검사 라인 삼성전자 日서 첫 설치 검토 https://www.mk.co.kr/article/10701618


In [7]:
import requests
from bs4 import BeautifulSoup

# 크롤링할 사이트 주소 정의 
source_url = "https://namu.news/section/news?page="
table_rows = []

# 페이지를 넘겨가면서 a 태그 크롤링 수행, list 만들어주기 
for page in range(5):
    req = requests.get(source_url+str(page+1))
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    contents_table = soup.find(name="div", attrs={"class":"xmbqsi-7"})
    table_rows.extend(contents_table.find_all(name="a"))

# a 태그의 href 속성을 리스트로 추출 
page_url_base = "https://namu.news"
page_urls = []
for row in table_rows:
    if len(row) > 0:
        page_url = page_url_base + row.get('href')
        page_urls.append(page_url)

# 중복 url 제거
page_urls = list(set(page_urls))

# 페이지 확인 
for page in page_urls:
    print(page)

In [13]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요:삼성전자

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):2

크롤링할 종료 페이지:  2 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&start=11']


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  7.36it/s]

검색된 기사 갯수: 총  20 개

[뉴스 제목]
["삼성전자, 세계 최초 '강아지용 스마트폰' 출시?", '삼성전자, 비스포크 냉장고 구매 고객에 패널 50% 할인', "'효자는 옛말' 반도체 수출 34%↓…삼성전자·SK하이닉스 셈법 복잡", "삼성전자, 7일 반도체 '어닝쇼크' 유력 [전기전자 1분기 실적 명암①]", '삼성전자 현직 부사장, 로봇 업체 레인보우로보틱스 합류', '삼성전자 "비스포크 냉장고 사면 패널 절반값에 드려요"', "'역시 기름집'...삼성전자보다 작년 연봉 높아", '일반 직원 평균연봉, 삼성전자보다 높은 곳은 어디?', '"예비신랑, 삼성전자에 물렸습니다"…결혼 앞둔 여성들 \'멘붕\' [박의명의 불개미 구조대]', '반도체 후공정 검사 라인 삼성전자 日서 첫 설치 검토', '반도체 보릿고개지만..."삼성전자, 직원 시급 12.5% 상승 검토 중"', "美반도체 보조금 신청 시작…삼성전자 '눈치' SK하이닉스 '여유'", '외인 삼성전자 공격 매수할까', '시스템에어컨 ·빌트인이 효자…LG전자 13년만 삼성 영업이익 추월할 듯', '美 마이크론 효과…삼성전자·SK하이닉스 강세', '효성티앤씨, 폐페트병으로 삼성전자 반도체 방진복 만든다', '"삼성전자, 목표가 7만→8만…하반기가 상반기보다 낫다"-IBK', "변리사가 가장 선호하는 기업은 '삼성전자'...꼴찌는", "삼성전자, 49만원대 가성비 폰 '갤럭시A34' 31일 출시", '"올해 두번 접는 폴더블폰 나온다"...삼성전자 특허 속 \'트리폴드\' 모습은']

[뉴스 링크]
['https://n.news.naver.com/mnews/article/018/0005454118?sid=101', 'https://n.news.naver.com/mnews/article/001/0013850843?sid=101', 'https://n.news.naver.com/mnews/article/448/0000402673?sid=101', 'https://n.news.naver.com/mnews

In [14]:
from konlpy.tag import *

hannanum = Hannanum()
result = ' '.join(s for s in news_contents)
hannanum.nouns(result)

['만우절',
 '홈페이지',
 '이벤트',
 '게시물',
 '올려[이데일리',
 '안혜',
 '기',
 '삼성전자(005930)',
 '세계',
 '최초',
 '‘강아지용',
 '스마트폰’',
 '출시',
 '1일',
 '삼성전자',
 '홈페이지',
 '‘갤럭시',
 'S23',
 '바우-와우',
 '에디션(Galaxy',
 'S23',
 'Edition)’',
 '출시',
 '게시물',
 '실제',
 '출시',
 '만우절',
 '이벤트성',
 '것',
 '이벤트성',
 '페이지',
 '상당한',
 '공',
 '‘고퀄(고퀄리티)’',
 '자랑',
 '삼성전자',
 '홈페이지',
 '캡쳐색상',
 '‘시고르자브종(시골잡종)',
 '크림색’',
 '가지',
 '이번',
 '에디션',
 '강아지용',
 '‘견생',
 '사진’',
 '수',
 '것',
 '한장',
 '체성분',
 '분석',
 '스트레스',
 '뿐',
 '관절염',
 '구',
 '녹내장',
 '등',
 '질병',
 '진단',
 '가능',
 '강아지',
 '혓바닥',
 '내밀기만해',
 '셀피',
 '수',
 '이',
 '강아지',
 '발바닥',
 '젤리',
 '스크린',
 '신체',
 '영양',
 '진단',
 '가능',
 '인공지능(AI)',
 '이',
 '분석',
 '강아지별',
 '맞춤형',
 '식단',
 '추천',
 '이번',
 '에디션',
 '핵심',
 '강아지',
 '코무늬,',
 '일병',
 '‘비문’',
 '해제',
 '가능',
 '점',
 '코끝',
 '터치',
 '한번',
 '정확',
 '잠금',
 '해제',
 '수',
 '강아지',
 '멋내기',
 '용품',
 '선글라스',
 '댕댕슈즈',
 '착용시',
 '얼굴',
 '식',
 '발바닥',
 '잠금해제',
 '점',
 '착안',
 '것',
 '삼성전자',
 '홈페이지',
 '캡쳐강아지',
 '집사들',
 '체력',
 '갤럭시',
 'S23',
 '바우-와우',
 '에디션',
 '놀',
 '모드',
 '지원',
 '이',
 '집사',